In [70]:
#!pip install -U python-dotenv
#!pip install spotipy
#!pip install matplotlib

     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     - -------------------------------------- 0.3/7.6 MB 5.2 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/7.6 MB 11.4 MB/s eta 0:00:01
     -------- ------------------------------- 1.7/7.6 MB 15.2 MB/s eta 0:00:01
     ----------- ---------------------------- 2.1/7.6 MB 11.3 MB/s eta 0:00:01
     ----------------- ---------------------- 3.4/7.6 MB 14.4 MB/s eta 0:00:01
     ---------------------- ----------------- 4.3/7.6 MB 15.3 MB/s eta 0:00:01
     --------------------------- ------------ 5.3/7.6 MB 16.2 MB/s eta 0:00:01
     -------------------------------- ------- 6.3/7.6 MB 16.7 MB/s eta 0:00:01
     ---------------------------------------  7.5/7.6 MB 17.9 MB/s eta 0:00:01
     ---------------------------------------  7.6/7.6 MB 18.1 MB/s eta 0:00:01
     ---------------------------------------- 7.6/7.6 MB 15.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/56.1 kB ? e


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\Jessi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import os
import plotly.express as px
import spotipy
from spotipy import oauth2
from dotenv import load_dotenv
import os
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
path = os.getcwd()

# setting up authentication for spotipy
load_dotenv(dotenv_path=path + f"\env_vars.env")
os.environ['SPOTIPY_CLIENT_ID'] = os.environ.get('SPOTIPY_CLIENT_ID')
os.environ['SPOTIPY_CLIENT_SECRET'] = os.environ.get('SPOTIPY_CLIENT_SECRET')
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

### Part 1: Loading in the Data
#### We will load in my personal data blah blah, you can replace the string with your file path to analyze your music

In [4]:
num_histories = 2
listening = pd.DataFrame()
for i in range(num_histories):
    temp_df = pd.read_json(path + f"\MyData\StreamingHistory{i}.json")
    listening = pd.concat([listening, temp_df])
print(listening.shape)
listening.head(5)

(18208, 4)


,endTime,artistName,trackName,msPlayed
0,2022-10-03 16:03,Tori Templet,Butterfly Rain,167166
1,2022-10-04 18:40,Maisie Peters,The List,68976
2,2022-10-07 18:05,Catie Turner,Nothing,4335
3,2022-10-07 18:05,Billie Marten,Mice,1650
4,2022-10-07 18:06,Jacob Collier,In Too Deep (feat. Kiana Ledé),18280


In [5]:
playlists = pd.read_json(path + f"\MyData\Playlist1.json")
playlists.head(5)

,playlists
0,"{'name': 'Pam and Jim Mix', 'lastModifiedDate'..."
1,{'name': 'Crying At The Laundrette - Strings V...
2,"{'name': 'Saved daily mix', 'lastModifiedDate'..."
3,"{'name': 'Hi', 'lastModifiedDate': '2023-08-05..."
4,"{'name': 'NT@B Car Vibes', 'lastModifiedDate':..."


### Part 2: EDA

In [6]:
playlists['playlists'][3]

{'name': 'Hi',
 'lastModifiedDate': '2023-08-05',
 'items': [{'track': {'trackName': 'What A Wonderful World',
    'artistName': "Israel Kamakawiwo'ole",
    'albumName': 'Wonderful World',
    'trackUri': 'spotify:track:1Fzm9s6Fh1Eumj5tU4q20m'},
   'episode': None,
   'localTrack': None,
   'addedDate': '2023-08-05'},
  {'track': {'trackName': 'A Thousand Years',
    'artistName': 'The Piano Guys',
    'albumName': 'The Piano Guys',
    'trackUri': 'spotify:track:4eYaDRhiL5iesFp2EuoODr'},
   'episode': None,
   'localTrack': None,
   'addedDate': '2023-08-05'}],
 'description': None,
 'numberOfFollowers': 0}

In [9]:
# information based on a single song
track_name, track_artist = 'Hey There Delilah', 'Plain White T\'s'
track_uri = sp.search(q='artist:' + track_artist + ' track:' + track_name, type='track')['tracks']['items'][0]['uri']

# extended_features = sp.audio_analysis(track_uri) # a LOT of information
# extended_features

track_features = sp.audio_features(track_uri)
track_features[0]

{'danceability': 0.656,
 'energy': 0.291,
 'key': 2,
 'loudness': -10.572,
 'mode': 1,
 'speechiness': 0.0293,
 'acousticness': 0.872,
 'instrumentalness': 0,
 'liveness': 0.114,
 'valence': 0.298,
 'tempo': 103.971,
 'type': 'audio_features',
 'id': '4RCWB3V8V0dignt99LZ8vH',
 'uri': 'spotify:track:4RCWB3V8V0dignt99LZ8vH',
 'track_href': 'https://api.spotify.com/v1/tracks/4RCWB3V8V0dignt99LZ8vH',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4RCWB3V8V0dignt99LZ8vH',
 'duration_ms': 232533,
 'time_signature': 4}

In [10]:
counts = listening.query('msPlayed >= 3000').groupby(by=['trackName', 'artistName']).size().reset_index()
songs_df = counts.rename(columns={0: 'count'}).sample(n=500, weights='count', random_state=42)#.query('count >= 5').sort_values('count', ascending=False) # either listened to >5 times or top x songs or random x songs
songs_df.head(5)#.to_csv('csv_exports/grouped.csv')

,trackName,artistName,count
1570,If By Chance,Ruth B.,1
4115,please,Chelsea Cutler,68
3207,"The Sun Will Come Up, The Seasons Will Change",Nina Nesbitt,21
2577,Rain,Kate Vogel,1
696,Dear August,PJ Harding,22


In [11]:

lst_of_cols = list(track_features[0].keys())
# lst_of_cols.remove(['type', 'analysis_url'])
# lst_of_cols.remove(x for x in ['type', 'analysis_url'])
[lst_of_cols.remove(col) for col in ['type', 'uri', 'analysis_url', 'track_href']]
def find_info(song):
    # print(song)
    try:
        song_dict = sp.search(q='artist:' + song[1] + ' track:' + song[0], type='track')
        song_uri = song_dict['tracks']['items'][0]['uri']
        track_features = sp.audio_features(song_uri)[0]
        return pd.Series([track_features[i] for i in lst_of_cols])
    except:
        return pd.Series([None] * len(lst_of_cols))
    
    
songs_df[lst_of_cols] = songs_df[['trackName', 'artistName']].apply(find_info, axis=1)
print(songs_df.shape)
songs_df.head(5)

(500, 17)


,trackName,artistName,count,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
1570,If By Chance,Ruth B.,1,0.405,0.279,10.0,-8.099,1.0,0.0334,0.881,0.0000,0.0933,0.256,119.850,21dpBHPTV7ythCHBI6Slcw,303653.0,4.0
4115,please,Chelsea Cutler,68,0.718,0.166,6.0,-15.820,0.0,0.0344,0.887,0.0216,0.1310,0.237,97.947,0TzeZppPwhEhySYaNRulr9,219067.0,4.0
3207,"The Sun Will Come Up, The Seasons Will Change",Nina Nesbitt,21,0.690,0.582,5.0,-6.259,0.0,0.0333,0.358,0.0000,0.0776,0.448,122.020,3i0VfLdOUCks9DmAht65XI,176853.0,4.0
2577,Rain,Kate Vogel,1,0.548,0.270,5.0,-10.200,1.0,0.0287,0.913,0.0432,0.0943,0.213,76.908,57uCYwWk48K38oIvozWR2P,150073.0,4.0
696,Dear August,PJ Harding,22,0.643,0.324,0.0,-7.817,1.0,0.0338,0.800,0.0000,0.1110,0.444,147.019,16gW9O8A4rIMx5OOVlzsSn,200940.0,4.0


### PCA attempt/plotting the PCs/scree plot

In [13]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [14]:
cleaned_df = songs_df.set_index('trackName').drop(columns=['id', 'artistName']).dropna()
centered_df = (cleaned_df - np.mean(cleaned_df, axis=0)) / np.std(cleaned_df, axis=0)
print(centered_df.shape)
centered_df.head(5)

(485, 14)


,count,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
trackName,,,,,,,,,,,,,,
If By Chance,-0.794261,-1.111479,-0.361044,1.489043,0.685850,0.400359,-0.438727,0.743378,-0.225513,-0.557376,-0.444100,0.112556,2.401944,0.256026
please,4.253560,1.387086,-1.012504,0.356202,-1.287643,-2.497760,-0.407398,0.765224,0.011026,-0.168989,-0.560302,-0.612775,0.547075,0.256026
"The Sun Will Come Up, The Seasons Will Change",0.712551,1.163572,1.385792,0.072992,1.156156,-2.497760,-0.441860,-1.160852,-0.225513,-0.719117,0.730149,0.184417,-0.378626,0.256026
Rain,-0.794261,0.030038,-0.412930,0.072992,0.148833,0.400359,-0.585973,0.859890,0.247565,-0.547073,-0.707083,-1.309495,-0.965879,0.256026
Dear August,0.787892,0.788388,-0.101613,-1.343058,0.757930,0.400359,-0.426196,0.448459,-0.225513,-0.375030,0.705686,1.012274,0.149572,0.256026


In [15]:
u, s, vt = np.linalg.svd(centered_df, full_matrices = False)
u.shape, s.shape, vt.shape

((485, 14), (14,), (14, 14))

In [16]:
np.round(s**2 / sum(s**2), 2)

array([0.2 , 0.11, 0.09, 0.08, 0.08, 0.07, 0.07, 0.06, 0.06, 0.06, 0.05,
       0.04, 0.03, 0.01])

In [17]:
# scree plot
px.line(s**2)

In [18]:
pcs = u * s
tst = centered_df.reset_index()[['trackName']]
tst['pc1'] = pcs[:,0]
tst['pc2'] = pcs[:,1]
tst['pc3'] = pcs[:,2]
px.scatter_3d(data_frame=tst, x='pc1', y='pc2', z='pc3', hover_data='trackName')
# pd.Index(cleaned_df).to_list()
# type(centered_df.reset_index()['id'])

In [19]:
# print(pcs)
all_pcs = pd.DataFrame(pcs)
all_pcs['trackName'] = centered_df.reset_index()[['trackName']]
all_pcs.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,trackName
0,0.562106,-1.956532,-0.499256,-1.712765,0.264985,-0.415133,-0.596594,0.402811,0.180296,1.091767,1.162425,-0.818803,-0.770142,-0.262920,If By Chance
1,1.716795,1.632879,0.743871,-1.625648,-2.470725,0.421683,0.570171,-2.321454,0.865630,-2.653123,1.395707,0.023597,0.448825,0.394301,please
2,-2.401405,0.611247,0.505239,-1.445142,-1.203955,0.643574,0.603206,-0.387061,-0.905500,-1.526107,-0.463916,-0.008417,-0.005958,0.154973,"The Sun Will Come Up, The Seasons Will Change"
3,0.955158,-0.041029,-1.383696,0.411642,-0.317086,-0.725038,0.137847,0.357598,-0.738751,-0.036598,-0.779370,0.276675,-0.800095,0.064894,Rain
4,-0.523112,-0.092174,0.688658,0.991783,-0.968944,0.942885,0.092613,0.729739,0.361996,-0.611669,0.895007,0.268943,-0.654240,-0.245951,Dear August


### Part 3: Analyzing/clustering the Data
#### PCA/plot/etc, figuring out what matters most in a song, what makes a song similar to another?

In [20]:
cluster_df = songs_df.set_index(['trackName', 'artistName', 'id']).dropna()
cluster_df = (cluster_df - np.mean(cluster_df, axis=0)) / np.std(cluster_df, axis=0)
cluster_df.head(5)

,,,count,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
trackName,artistName,id,,,,,,,,,,,,,,
If By Chance,Ruth B.,21dpBHPTV7ythCHBI6Slcw,-0.794261,-1.111479,-0.361044,1.489043,0.685850,0.400359,-0.438727,0.743378,-0.225513,-0.557376,-0.444100,0.112556,2.401944,0.256026
please,Chelsea Cutler,0TzeZppPwhEhySYaNRulr9,4.253560,1.387086,-1.012504,0.356202,-1.287643,-2.497760,-0.407398,0.765224,0.011026,-0.168989,-0.560302,-0.612775,0.547075,0.256026
"The Sun Will Come Up, The Seasons Will Change",Nina Nesbitt,3i0VfLdOUCks9DmAht65XI,0.712551,1.163572,1.385792,0.072992,1.156156,-2.497760,-0.441860,-1.160852,-0.225513,-0.719117,0.730149,0.184417,-0.378626,0.256026
Rain,Kate Vogel,57uCYwWk48K38oIvozWR2P,-0.794261,0.030038,-0.412930,0.072992,0.148833,0.400359,-0.585973,0.859890,0.247565,-0.547073,-0.707083,-1.309495,-0.965879,0.256026
Dear August,PJ Harding,16gW9O8A4rIMx5OOVlzsSn,0.787892,0.788388,-0.101613,-1.343058,0.757930,0.400359,-0.426196,0.448459,-0.225513,-0.375030,0.705686,1.012274,0.149572,0.256026


In [21]:
# import scikit-learn as sklearn
from sklearn.cluster import MeanShift
from sklearn.cluster import KMeans

In [22]:
cluster = MeanShift().fit(cluster_df)
cluster.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 8, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,

In [23]:
cluster_w_labels = cluster_df.reset_index()
cluster_w_labels['color'] = cluster.labels_
# px.scatter(cluster_w_labels, x='count', y='energy', color='color', hover_data='trackName')
# cluster_w_labels.sort_values('color', ascending=False)

In [24]:
cluster = KMeans(n_clusters=cluster_df.shape[0] // 10).fit_predict(cluster_df)
cluster_w_labels = cluster_df.reset_index()
cluster_w_labels['color'] = cluster
px.scatter(cluster_w_labels, x='count', y='energy', color='color', hover_data='trackName')

C:\Users\Jessi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\Jessi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning:

Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "C:\Users\Jessi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_p

In [25]:
cluster = KMeans(n_clusters=tst.shape[0] // 10, n_init='auto').fit_predict(tst.drop(columns=['trackName']))
tst_plot = tst.reset_index()
tst_plot['color'] = cluster
px.scatter_3d(tst_plot, x='pc1', y='pc2', z='pc3', hover_data='trackName', color='color')

In [26]:
all_pc_cluster = KMeans(n_clusters=all_pcs.shape[0] // 10, n_init='auto').fit_predict(all_pcs.drop(columns=['trackName'])) #all_pcs
all_pc_plot = all_pcs
all_pc_plot['color'] = all_pc_cluster
px.scatter_3d(all_pc_plot, x=0, y=1, z=2, hover_data='trackName', color='color')

In [39]:
def get_playlist(song):
    label = cluster_w_labels[cluster_w_labels['trackName'] == song].color.iloc[0] #.loc[0, 'color']
    playlist = cluster_w_labels[cluster_w_labels['color'] == label]
    return playlist['trackName']

    # # this is using the 3 PCs instead of raw elements, current least favorite bc it's less relevant
    # label = tst_plot[tst_plot['trackName'] == song].iloc[0, -1]#.loc['color', 0]
    # playlist = tst_plot[tst_plot['color'] == label]
    # return playlist['trackName']

    # this uses all the PCs instead of raw or 3 PCs
    # label = all_pc_plot[all_pc_plot['trackName'] == song].iloc[0, -1]#.loc['color', 0]
    # playlist = all_pc_plot[all_pc_plot['color'] == label]
    # return playlist['trackName']
    

In [42]:
# get_playlist('By Your Side')
# get_playlist('I Lived')
# get_playlist('Hey It\'s Delilah')
get_playlist('Hey There Delilah')

10                           the author
17        Letting Go for a Little While
20     Crazier Things (with Noah Kahan)
63                         Better Place
73                               broken
74               Safe Places (Acoustic)
127                   Hey There Delilah
139                           Soul Mate
157                               Crave
158                              I Knew
175                           Seventeen
195                               Paint
213                  A Little Bit Yours
231          Where's My Love - Acoustic
245                           Haunt You
258                                  OK
292                           Same Boat
295                    Begging for More
304                      In Your Circle
308                                Bees
329         Vanilla (Stairwell Version)
347                To Love Someone Else
358                           Turns Out
364                 Bloom - Bonus Track
383                     Pictures of You


### Part 3: Loading in a playlist and comparing it to other songs in the playlist
#### Read in, PCA, plot with colors

### Part 4: Selecting additional relevant songs for playlist
#### Find most similar ones to playlist songs